##**EXTRACCION DE DATOS DE KAGGLE**

In [24]:
import os
from google.colab import files
files.upload()  # Se crea el token , se descarga y aquí se sube el kaggle.json

os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p /content
!unzip -o /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -d /content/udea_saber_pro




Saving kaggle.json to kaggle.json
udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: /content/udea_saber_pro/submission_example.csv  
  inflating: /content/udea_saber_pro/test.csv  
  inflating: /content/udea_saber_pro/train.csv  


##**LIBRERIAS PROYECTO**

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from IPython.display import display
pd.set_option('display.max_columns', None)

##**Cargar Datos:**

In [26]:
# Cargar datos
train = pd.read_csv("/content/udea_saber_pro/train.csv")
test = pd.read_csv("/content/udea_saber_pro/test.csv")
sample = pd.read_csv("/content/udea_saber_pro/submission_example.csv")

# Verificar
print("Train:", train.shape)
print("Test:", test.shape)
print("Sample submission:", sample.shape)

Train: (692500, 21)
Test: (296786, 20)
Sample submission: (296786, 2)


In [27]:
# Datos que se han cargado de entrenamiento
train.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [28]:
# Resumen de la información del DataFrame
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [32]:
# Descripción del DataFrame de las columnas numéricas
train.describe()

,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,0.268629,0.259996,0.262087,0.262903
std,285585.209455,10.535037,0.122130,0.093480,0.058862,0.067944
min,1.000000,20183.000000,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,0.203000,0.212000,0.254000,0.255000
50%,494564.500000,20195.000000,0.240000,0.271000,0.276000,0.285000
75%,741782.500000,20203.000000,0.314000,0.309000,0.293000,0.303000
max,989286.000000,20213.000000,0.657000,0.487000,0.320000,0.332000


##**FUNCIONES**

In [30]:
# Tabla resumen
# train.info() con un estilo aplicado
def resumen(dataframe):
  # Cálculos de resumen
  null_counts = dataframe.isnull().sum()
  total_counts = len(dataframe)
  non_null_counts = dataframe.notnull().sum()
  data_types = dataframe.dtypes
  unique_counts = dataframe.nunique()

  # Crear DataFrame de resumen
  summary_df = pd.DataFrame({
      'Tipo de Dato': data_types,
      'Valores Únicos': unique_counts,
      'No Nulos': non_null_counts,
      'Nulos': null_counts,
      '% Nulos': (null_counts / total_counts) * 100
  })

  # Ordenar por % de nulos
  summary_df = summary_df.sort_values(by='% Nulos', ascending=False)

  # Aplicar formato con Styler
  styled_summary = summary_df.style \
      .format({'% Nulos': '{:.2f}%'}) \
      .bar(subset='% Nulos', color='#f08080') \
      .set_properties(**{'text-align': 'center'}) \
      .set_caption("Resumen de columnas del DataFrame") \
      .set_table_styles([
          # Color del encabezado de columnas
          {'selector': 'th', 'props': [
              ('text-align', 'center'),
              ('background-color', '#1f4e79'),
              ('color', 'white'),
              ('font-weight', 'bold')
          ]},
          # Color de las celdas
          {'selector': 'td', 'props': [('padding', '5px')]},
          # Estilo del título de la tabla
          {'selector': 'caption', 'props': [
              ('color', '#1f4e79'),
              ('font-size', '18px'),
              ('text-align', 'center'),
              ('font-weight', 'bold')
          ]}
      ])

  # Mostrar
  display(styled_summary)

  # Información final
  print(f"\n El DataFrame tiene un total de **{len(dataframe.columns)}** columnas y **{len(dataframe)}** filas.")

In [31]:
#Función Encabezado
# train.head() con un estilo aplicado
def encabezado(dataframe,filas=5):
  salida=dataframe.head(filas).style \
      .set_properties(**{
          'background-color': '#f9f9f9',
          'border': '1px solid #ddd',
          'text-align': 'center'
      }) \
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', '#4B8BBE'), ('color', 'white'), ('text-align', 'center')]}
      ]) \
      .highlight_null() \
      .set_caption("Vista Previa de los Primeros Registros")
  display(salida)

##**Visualización**

###**Datos de Entrenamiento**

####**Resumen**

In [68]:
#Tabla resumen de las columnas de entrenamiento
resumen(train)

,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
F_TIENEAUTOMOVIL,object,2,648877,43623,6.30%
F_TIENELAVADORA,object,2,652727,39773,5.74%
F_TIENECOMPUTADOR,object,2,654397,38103,5.50%
F_ESTRATOVIVIENDA,object,7,660363,32137,4.64%
E_HORASSEMANATRABAJA,object,5,661643,30857,4.46%
F_TIENEINTERNET.1,object,2,665871,26629,3.85%
F_TIENEINTERNET,object,2,665871,26629,3.85%
F_EDUCACIONMADRE,object,12,668836,23664,3.42%
F_EDUCACIONPADRE,object,12,669322,23178,3.35%
E_PAGOMATRICULAPROPIO,object,2,686002,6498,0.94%



 El DataFrame tiene un total de **21** columnas y **692500** filas.


In [69]:
# Muestra de datos de entrenamiento
encabezado(train)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322000,0.208000,0.310000,0.267000
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311000,0.215000,0.292000,0.264000
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297000,0.214000,0.305000,0.264000
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485000,0.172000,0.252000,0.190000
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316000,0.232000,0.285000,0.294000


####**Columnas Binarias**

In [59]:
visualizacion_datos(train,"F_TIENEINTERNET")

,Frecuencia,% del Total
F_TIENEINTERNET,,
Si,592514,85.56%
No,73357,10.59%
Desconocido,26629,3.85%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEINTERNET,,,,
Desconocido,5981,8382,6301,5965
No,8480,28916,14944,21017
Si,161158,135689,150374,145293



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEINTERNET,,,,
Desconocido,22.46%,31.48%,23.66%,22.40%
No,11.56%,39.42%,20.37%,28.65%
Si,27.20%,22.90%,25.38%,24.52%


In [60]:
visualizacion_datos(train,"F_TIENELAVADORA")

,Frecuencia,% del Total
F_TIENELAVADORA,,
Si,563390,81.36%
No,89337,12.90%
Desconocido,39773,5.74%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENELAVADORA,,,,
Desconocido,7882,13397,9166,9328
No,13966,30554,20167,24650
Si,153771,129036,142286,138297



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENELAVADORA,,,,
Desconocido,19.82%,33.68%,23.05%,23.45%
No,15.63%,34.20%,22.57%,27.59%
Si,27.29%,22.90%,25.26%,24.55%


In [61]:
visualizacion_datos(train,"F_TIENEAUTOMOVIL")

,Frecuencia,% del Total
F_TIENEAUTOMOVIL,,
No,412606,59.58%
Si,236271,34.12%
Desconocido,43623,6.30%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEAUTOMOVIL,,,,
Desconocido,8402,14889,9970,10362
No,85543,114464,101910,110689
Si,81674,43634,59739,51224



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEAUTOMOVIL,,,,
Desconocido,19.26%,34.13%,22.85%,23.75%
No,20.73%,27.74%,24.70%,26.83%
Si,34.57%,18.47%,25.28%,21.68%


In [62]:
visualizacion_datos(train,"E_PRIVADO_LIBERTAD")

,Frecuencia,% del Total
E_PRIVADO_LIBERTAD,,
N,692466,100.00%
S,34,0.00%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRIVADO_LIBERTAD,,,,
N,175613,172974,171613,172266
S,6,13,6,9



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRIVADO_LIBERTAD,,,,
N,25.36%,24.98%,24.78%,24.88%
S,17.65%,38.24%,17.65%,26.47%


In [63]:
visualizacion_datos(train,"E_PAGOMATRICULAPROPIO")

,Frecuencia,% del Total
E_PAGOMATRICULAPROPIO,,
No,382201,55.19%
Si,303801,43.87%
Desconocido,6498,0.94%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PAGOMATRICULAPROPIO,,,,
Desconocido,2332,1277,1541,1348
No,123072,76324,97507,85298
Si,50215,95386,72571,85629



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PAGOMATRICULAPROPIO,,,,
Desconocido,35.89%,19.65%,23.71%,20.74%
No,32.20%,19.97%,25.51%,22.32%
Si,16.53%,31.40%,23.89%,28.19%


In [64]:
visualizacion_datos(train,"F_TIENECOMPUTADOR")

,Frecuencia,% del Total
F_TIENECOMPUTADOR,,
Si,597670,86.31%
No,56727,8.19%
Desconocido,38103,5.50%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENECOMPUTADOR,,,,
Desconocido,7663,12793,8805,8842
No,6072,23128,11321,16206
Si,161884,137066,151493,147227



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENECOMPUTADOR,,,,
Desconocido,20.11%,33.57%,23.11%,23.21%
No,10.70%,40.77%,19.96%,28.57%
Si,27.09%,22.93%,25.35%,24.63%


In [65]:
visualizacion_datos(train,"F_TIENEINTERNET.1")

,Frecuencia,% del Total
F_TIENEINTERNET.1,,
Si,592514,85.56%
No,73357,10.59%
Desconocido,26629,3.85%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEINTERNET.1,,,,
Desconocido,5981,8382,6301,5965
No,8480,28916,14944,21017
Si,161158,135689,150374,145293



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_TIENEINTERNET.1,,,,
Desconocido,22.46%,31.48%,23.66%,22.40%
No,11.56%,39.42%,20.37%,28.65%
Si,27.20%,22.90%,25.38%,24.52%


####**Columnas no Binarias**

In [66]:
visualizacion_datos(train,"PERIODO_ACADEMICO")

,Frecuencia,% del Total
PERIODO_ACADEMICO,,
20195,180873,26.12%
20203,171838,24.81%
20212,171412,24.75%
20183,164818,23.80%
20194,1472,0.21%
20213,1178,0.17%
20202,490,0.07%
20184,254,0.04%
20196,165,0.02%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
PERIODO_ACADEMICO,,,,
20183,44533,35650,41720,42915
20184,58,72,53,71
20194,816,88,357,211
20195,41275,45253,45665,48680
20196,31,33,54,47
20202,257,48,113,72
20203,48648,40543,43612,39035
20212,39488,51103,39763,41058
20213,513,197,282,186



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
PERIODO_ACADEMICO,,,,
20183,27.02%,21.63%,25.31%,26.04%
20184,22.83%,28.35%,20.87%,27.95%
20194,55.43%,5.98%,24.25%,14.33%
20195,22.82%,25.02%,25.25%,26.91%
20196,18.79%,20.00%,32.73%,28.48%
20202,52.45%,9.80%,23.06%,14.69%
20203,28.31%,23.59%,25.38%,22.72%
20212,23.04%,29.81%,23.20%,23.95%
20213,43.55%,16.72%,23.94%,15.79%


In [70]:
visualizacion_datos(train,"E_PRGM_ACADEMICO")

,Frecuencia,% del Total
E_PRGM_ACADEMICO,,
DERECHO,53244,7.69%
ADMINISTRACION DE EMPRESAS,51902,7.49%
CONTADURIA PUBLICA,39664,5.73%
PSICOLOGIA,31932,4.61%
INGENIERIA INDUSTRIAL,28481,4.11%
ADMINISTRACIÓN DE EMPRESAS,24777,3.58%
INGENIERIA CIVIL,19487,2.81%
MEDICINA,18099,2.61%
PSICOLOGÍA,17377,2.51%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRGM_ACADEMICO,,,,
3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,11,21,16,21
ACTIVIDAD FISICA Y DEPORTE,13,68,33,48
ACUICULTURA,0,45,7,27
ADMINISTRACION,24,30,35,28
ADMINISTRACION FINANCIERA,13,25,13,23
ADMINISTRACION & SERVICIO,26,0,3,0
ADMINISTRACION AERONAUTICA,23,7,31,17
ADMINISTRACION AGROPECUARIA,4,82,6,25
ADMINISTRACION AMBIENTAL,187,100,141,94



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRGM_ACADEMICO,,,,
3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,15.94%,30.43%,23.19%,30.43%
ACTIVIDAD FISICA Y DEPORTE,8.02%,41.98%,20.37%,29.63%
ACUICULTURA,0.00%,56.96%,8.86%,34.18%
ADMINISTRACION,20.51%,25.64%,29.91%,23.93%
ADMINISTRACION FINANCIERA,17.57%,33.78%,17.57%,31.08%
ADMINISTRACION & SERVICIO,89.66%,0.00%,10.34%,0.00%
ADMINISTRACION AERONAUTICA,29.49%,8.97%,39.74%,21.79%
ADMINISTRACION AGROPECUARIA,3.42%,70.09%,5.13%,21.37%
ADMINISTRACION AMBIENTAL,35.82%,19.16%,27.01%,18.01%


In [71]:
visualizacion_datos(train,"E_PRGM_DEPARTAMENTO")

,Frecuencia,% del Total
E_PRGM_DEPARTAMENTO,,
BOGOTÁ,282159,40.74%
ANTIOQUIA,83607,12.07%
VALLE,44588,6.44%
ATLANTICO,41020,5.92%
SANTANDER,28828,4.16%
NORTE SANTANDER,22588,3.26%
BOLIVAR,20629,2.98%
BOYACA,14048,2.03%
CUNDINAMARCA,14018,2.02%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRGM_DEPARTAMENTO,,,,
AMAZONAS,1,17,6,16
ANTIOQUIA,25688,17217,21657,19045
ARAUCA,33,283,92,181
ATLANTICO,10165,11159,9484,10212
BOGOTÁ,77388,66721,69215,68835
BOLIVAR,3816,6577,4758,5478
BOYACA,3770,2338,4440,3500
CALDAS,3751,1916,3313,2660
CAQUETA,283,979,568,829



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_PRGM_DEPARTAMENTO,,,,
AMAZONAS,2.50%,42.50%,15.00%,40.00%
ANTIOQUIA,30.72%,20.59%,25.90%,22.78%
ARAUCA,5.60%,48.05%,15.62%,30.73%
ATLANTICO,24.78%,27.20%,23.12%,24.90%
BOGOTÁ,27.43%,23.65%,24.53%,24.40%
BOLIVAR,18.50%,31.88%,23.06%,26.55%
BOYACA,26.84%,16.64%,31.61%,24.91%
CALDAS,32.23%,16.46%,28.46%,22.85%
CAQUETA,10.64%,36.82%,21.36%,31.18%


In [72]:
visualizacion_datos(train,"E_VALORMATRICULAUNIVERSIDAD")

,Frecuencia,% del Total
E_VALORMATRICULAUNIVERSIDAD,,
Entre 1 millón y menos de 2.5 millones,204048,29.47%
Entre 2.5 millones y menos de 4 millones,127430,18.40%
Menos de 500 mil,80263,11.59%
Entre 500 mil y menos de 1 millón,78704,11.37%
Entre 4 millones y menos de 5.5 millones,69736,10.07%
Más de 7 millones,68014,9.82%
Entre 5.5 millones y menos de 7 millones,38490,5.56%
No pagó matrícula,19528,2.82%
Desconocido,6287,0.91%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_VALORMATRICULAUNIVERSIDAD,,,,
Desconocido,2289,1219,1489,1290
Entre 1 millón y menos de 2.5 millones,26775,72116,44287,60870
Entre 2.5 millones y menos de 4 millones,22181,33713,34543,36993
Entre 4 millones y menos de 5.5 millones,20032,11707,21212,16785
Entre 5.5 millones y menos de 7 millones,14881,4648,11360,7601
Entre 500 mil y menos de 1 millón,16382,22522,19121,20679
Menos de 500 mil,23977,17988,21030,17268
Más de 7 millones,43640,3690,13988,6696
No pagó matrícula,5462,5384,4589,4093



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_VALORMATRICULAUNIVERSIDAD,,,,
Desconocido,36.41%,19.39%,23.68%,20.52%
Entre 1 millón y menos de 2.5 millones,13.12%,35.34%,21.70%,29.83%
Entre 2.5 millones y menos de 4 millones,17.41%,26.46%,27.11%,29.03%
Entre 4 millones y menos de 5.5 millones,28.73%,16.79%,30.42%,24.07%
Entre 5.5 millones y menos de 7 millones,38.66%,12.08%,29.51%,19.75%
Entre 500 mil y menos de 1 millón,20.81%,28.62%,24.29%,26.27%
Menos de 500 mil,29.87%,22.41%,26.20%,21.51%
Más de 7 millones,64.16%,5.43%,20.57%,9.85%
No pagó matrícula,27.97%,27.57%,23.50%,20.96%


In [73]:
visualizacion_datos(train,"E_HORASSEMANATRABAJA")

,Frecuencia,% del Total
E_HORASSEMANATRABAJA,,
Más de 30 horas,249352,36.01%
0,116550,16.83%
Entre 11 y 20 horas,115857,16.73%
Entre 21 y 30 horas,92693,13.39%
Menos de 10 horas,87191,12.59%
Desconocido,30857,4.46%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_HORASSEMANATRABAJA,,,,
0,45754,18959,28900,22937
Desconocido,7038,9350,7492,6977
Entre 11 y 20 horas,27554,32019,27400,28884
Entre 21 y 30 horas,20926,24483,22968,24316
Menos de 10 horas,23358,25434,18757,19642
Más de 30 horas,50989,62742,66102,69519



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
E_HORASSEMANATRABAJA,,,,
0,39.26%,16.27%,24.80%,19.68%
Desconocido,22.81%,30.30%,24.28%,22.61%
Entre 11 y 20 horas,23.78%,27.64%,23.65%,24.93%
Entre 21 y 30 horas,22.58%,26.41%,24.78%,26.23%
Menos de 10 horas,26.79%,29.17%,21.51%,22.53%
Más de 30 horas,20.45%,25.16%,26.51%,27.88%


In [74]:
visualizacion_datos(train,"F_ESTRATOVIVIENDA")

,Frecuencia,% del Total
F_ESTRATOVIVIENDA,,
Estrato 2,232671,33.60%
Estrato 3,210685,30.42%
Estrato 1,111991,16.17%
Estrato 4,65514,9.46%
Desconocido,32137,4.64%
Estrato 5,23608,3.41%
Estrato 6,12605,1.82%
Sin Estrato,3289,0.47%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_ESTRATOVIVIENDA,,,,
Desconocido,6912,10257,7440,7528
Estrato 1,14363,42388,23643,31597
Estrato 2,45948,63068,59536,64119
Estrato 3,59121,42756,56841,51967
Estrato 4,28624,9064,16154,11672
Estrato 5,12415,2731,5112,3350
Estrato 6,7648,1287,2314,1356
Sin Estrato,588,1436,579,686



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_ESTRATOVIVIENDA,,,,
Desconocido,21.51%,31.92%,23.15%,23.42%
Estrato 1,12.83%,37.85%,21.11%,28.21%
Estrato 2,19.75%,27.11%,25.59%,27.56%
Estrato 3,28.06%,20.29%,26.98%,24.67%
Estrato 4,43.69%,13.84%,24.66%,17.82%
Estrato 5,52.59%,11.57%,21.65%,14.19%
Estrato 6,60.67%,10.21%,18.36%,10.76%
Sin Estrato,17.88%,43.66%,17.60%,20.86%


In [75]:
visualizacion_datos(train,"F_EDUCACIONPADRE")

,Frecuencia,% del Total
F_EDUCACIONPADRE,,
Secundaria (Bachillerato) completa,128289,18.53%
Primaria incompleta,125675,18.15%
Educación profesional completa,83117,12.00%
Secundaria (Bachillerato) incompleta,71654,10.35%
Técnica o tecnológica completa,62995,9.10%
Primaria completa,55958,8.08%
Postgrado,44169,6.38%
Educación profesional incompleta,27084,3.91%
Desconocido,23178,3.35%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_EDUCACIONPADRE,,,,
Desconocido,5071,7693,5293,5121
Educación profesional completa,33397,13733,20317,15670
Educación profesional incompleta,9473,5131,6879,5601
Ninguno,2091,9714,3961,6242
No Aplica,2325,2070,2455,2379
No sabe,5060,3068,4626,3838
Postgrado,23856,4508,9732,6073
Primaria completa,8838,17647,13359,16114
Primaria incompleta,16839,42457,28937,37442



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_EDUCACIONPADRE,,,,
Desconocido,21.88%,33.19%,22.84%,22.09%
Educación profesional completa,40.18%,16.52%,24.44%,18.85%
Educación profesional incompleta,34.98%,18.94%,25.40%,20.68%
Ninguno,9.50%,44.14%,18.00%,28.36%
No Aplica,25.19%,22.43%,26.60%,25.78%
No sabe,30.50%,18.49%,27.88%,23.13%
Postgrado,54.01%,10.21%,22.03%,13.75%
Primaria completa,15.79%,31.54%,23.87%,28.80%
Primaria incompleta,13.40%,33.78%,23.03%,29.79%


In [76]:
visualizacion_datos(train,"F_EDUCACIONMADRE")

,Frecuencia,% del Total
F_EDUCACIONMADRE,,
Secundaria (Bachillerato) completa,141744,20.47%
Primaria incompleta,99420,14.36%
Técnica o tecnológica completa,89542,12.93%
Educación profesional completa,85326,12.32%
Secundaria (Bachillerato) incompleta,81012,11.70%
Primaria completa,56125,8.10%
Postgrado,46246,6.68%
Técnica o tecnológica incompleta,27533,3.98%
Desconocido,23664,3.42%



 Tabla cruzada (cantidad):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_EDUCACIONMADRE,,,,
Desconocido,5172,7864,5393,5235
Educación profesional completa,35790,13210,20689,15637
Educación profesional incompleta,8232,4069,5714,4455
Ninguno,1116,7111,2210,4046
No Aplica,344,649,402,523
No sabe,761,824,728,704
Postgrado,23581,5001,10792,6872
Primaria completa,8117,18350,13244,16414
Primaria incompleta,11522,36434,21615,29849



 Tabla cruzada (porcentajes):



RENDIMIENTO_GLOBAL,alto,bajo,medio-alto,medio-bajo
F_EDUCACIONMADRE,,,,
Desconocido,21.86%,33.23%,22.79%,22.12%
Educación profesional completa,41.95%,15.48%,24.25%,18.33%
Educación profesional incompleta,36.64%,18.11%,25.43%,19.83%
Ninguno,7.71%,49.10%,15.26%,27.94%
No Aplica,17.94%,33.84%,20.96%,27.27%
No sabe,25.22%,27.31%,24.13%,23.33%
Postgrado,50.99%,10.81%,23.34%,14.86%
Primaria completa,14.46%,32.69%,23.60%,29.25%
Primaria incompleta,11.59%,36.65%,21.74%,30.02%


####**No Binarias Alta Cardinalidad**

In [89]:
valores_unicos = train['E_PRGM_ACADEMICO'].unique()
# Procesar y ordenar los valores
valores_procesados = sorted([str(valor).upper().replace(' ', '_') for valor in valores_unicos])
print(valores_procesados)

['3°__CICLO_PROFESIONAL_NEGOCIOS_INTERNACIONALES', 'ACTIVIDAD_FISICA_Y_DEPORTE', 'ACUICULTURA', 'ADMINISTRACION', 'ADMINISTRACION_&_SERVICIO', 'ADMINISTRACION_AERONAUTICA', 'ADMINISTRACION_AGROPECUARIA', 'ADMINISTRACION_AMBIENTAL', 'ADMINISTRACION_AMBIENTAL_Y_DE_LOS_RECURSOS_NATURALES', 'ADMINISTRACION_BANCARIA_Y_FINANCIERA', 'ADMINISTRACION_COMERCIAL', 'ADMINISTRACION_COMERCIAL_Y_DE_MERCADEO', 'ADMINISTRACION_DEL_COMERCIO_INTERNACIONAL', 'ADMINISTRACION_DEL_MEDIO_AMBIENTE', 'ADMINISTRACION_DEL_TURISMO_SOSTENIBLE', 'ADMINISTRACION_DEPORTIVA', 'ADMINISTRACION_DE_COMERCIO_EXTERIOR', 'ADMINISTRACION_DE_EMPRESAS', 'ADMINISTRACION_DE_EMPRESAS_AGROINDUSTRIALES', 'ADMINISTRACION_DE_EMPRESAS_AGROPECUARIAS', 'ADMINISTRACION_DE_EMPRESAS_COMERCIALES', 'ADMINISTRACION_DE_EMPRESAS_EN_TELECOMUNICACIONES', 'ADMINISTRACION_DE_EMPRESAS_TURISTICA', 'ADMINISTRACION_DE_EMPRESAS_TURISTICAS', 'ADMINISTRACION_DE_EMPRESAS_TURISTICAS_Y_HOTELERAS', 'ADMINISTRACION_DE_EMPRESAS_TURÍSTICAS_Y_HOTELERAS', 'ADMINISTR

####**Numéricas**

In [77]:
#Parametros Numéricos
new_train_num=train[["INDICADOR_1","INDICADOR_2","INDICADOR_3","INDICADOR_4"]]

In [78]:
encabezado(new_train_num)

,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,0.322000,0.208000,0.310000,0.267000
1,0.311000,0.215000,0.292000,0.264000
2,0.297000,0.214000,0.305000,0.264000
3,0.485000,0.172000,0.252000,0.190000
4,0.316000,0.232000,0.285000,0.294000


In [79]:
target=train["RENDIMIENTO_GLOBAL"]
target.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619
